In [1]:
import pandas as pd
import numpy as np

import scipy.stats as stats
from scipy.stats import kendalltau
from scipy.spatial.distance import pdist, squareform

from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from math import sqrt
import matplotlib.pyplot as plt
from pyEDM import *

import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

1.1. Import data

In [13]:
#############
TICKER = 2330
TP = 1
TARGET=f'y_{TP}'
#############

### import data ###
data = pd.read_csv('/Users/yitsung/Desktop/MasterThesis/data/TaiwanStockData_Top100_EMA')
ticker_data = data[data['ticker']==TICKER].reset_index(drop=True)
ticker_data = ticker_data.drop(columns=['ticker'])

### generate y ###
# ver.2(SMA-P/P, 2class) #
ticker_data[f'y_{TP}'] = ticker_data['close'].rolling(window=TP).mean()
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(-TP)
ticker_data = ticker_data.dropna().reindex()
ticker_data[f'y_{TP}'] = ((ticker_data[f'y_{TP}'] - ticker_data['close']) >= 0).astype(int)

# 對答案用#
origi_data = ticker_data.copy()

# 還原成不洩露資訊的df #
ticker_data[f'y_{TP}'] = ticker_data[f'y_{TP}'].shift(TP)
ticker_data = ticker_data.dropna().reindex()

ticker_data.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_1
703,2023-11-24,577.0,578.0,574.0,575.0,12503.0,243.0,-854.0,70.0,-118.0,-2263.0,2.8318,575.073961,572.538736,562.541337,9.997398,9.134503,90.744592,0.0
704,2023-11-27,573.0,577.0,568.0,568.0,20322.0,-112.0,-2153.0,59.0,-56.0,-3554.0,4.1507,573.659169,571.840469,562.945683,8.894786,9.086560,81.069290,0.0
705,2023-11-28,565.0,576.0,565.0,575.0,26932.0,478.0,3323.0,-98.0,687.0,-416.0,5.1624,573.927335,572.326550,563.838595,8.487955,8.966839,76.500832,1.0
706,2023-11-29,578.0,579.0,570.0,574.0,27787.0,357.0,-180.0,55.0,-553.0,-2383.0,4.8624,573.941868,572.584004,564.591292,7.992712,8.772014,71.301362,0.0
707,2023-11-30,576.0,577.0,570.0,577.0,54365.0,-32.0,4730.0,-68.0,-770.0,-155.0,7.5527,574.553494,573.263388,565.510455,7.752933,8.568197,68.146342,1.0


In [14]:
origi_data[origi_data['Date']>='2023-07-01']

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_1
603,2023-07-03,578.0,580.0,576.0,579.0,15118.0,97.0,2353.0,-305.0,1401.0,582.0,2.5477,576.701182,575.940219,566.756798,9.183421,11.833896,33.311646,1
604,2023-07-04,585.0,585.0,580.0,585.0,17777.0,84.0,4805.0,-1348.0,-13.0,1767.0,2.7068,578.360946,577.334031,568.108146,9.225886,11.312294,30.627646,0
605,2023-07-05,589.0,589.0,579.0,582.0,15554.0,-50.0,-890.0,-503.0,-1092.0,-790.0,2.6473,579.088757,578.051873,569.137172,8.914701,10.832775,17.707207,0
606,2023-07-06,573.0,574.0,565.0,565.0,32070.0,563.0,-16476.0,-574.0,-603.0,-14045.0,5.1447,576.271005,576.043892,568.830715,7.213177,10.108856,7.697299,1
607,2023-07-07,565.0,572.0,563.0,565.0,19859.0,32.0,-4486.0,-185.0,-100.0,-1681.0,3.6264,574.016804,574.344832,568.546958,5.797874,9.246659,1.662877,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,2023-11-24,577.0,578.0,574.0,575.0,12503.0,243.0,-854.0,70.0,-118.0,-2263.0,2.8318,575.073961,572.538736,562.541337,9.997398,9.134503,90.744592,0
704,2023-11-27,573.0,577.0,568.0,568.0,20322.0,-112.0,-2153.0,59.0,-56.0,-3554.0,4.1507,573.659169,571.840469,562.945683,8.894786,9.086560,81.069290,1
705,2023-11-28,565.0,576.0,565.0,575.0,26932.0,478.0,3323.0,-98.0,687.0,-416.0,5.1624,573.927335,572.326550,563.838595,8.487955,8.966839,76.500832,0
706,2023-11-29,578.0,579.0,570.0,574.0,27787.0,357.0,-180.0,55.0,-553.0,-2383.0,4.8624,573.941868,572.584004,564.591292,7.992712,8.772014,71.301362,1


1.2.Splite data into train(Library) and test(Prediction)

In [15]:
Library = ticker_data[ticker_data['Date'] <= '2023-06-30'] # windows=20, 最後預測到6/30
Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-07-20')] # test
# Prediction = ticker_data[(ticker_data['Date'] >= '2023-07-01')&(ticker_data['Date'] <= '2023-10-31')] 
Prediction

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_1
603,2023-07-03,578.0,580.0,576.0,579.0,15118.0,97.0,2353.0,-305.0,1401.0,582.0,2.5477,576.701182,575.940219,566.756798,9.183421,11.833896,33.311646,1.0
604,2023-07-04,585.0,585.0,580.0,585.0,17777.0,84.0,4805.0,-1348.0,-13.0,1767.0,2.7068,578.360946,577.334031,568.108146,9.225886,11.312294,30.627646,1.0
605,2023-07-05,589.0,589.0,579.0,582.0,15554.0,-50.0,-890.0,-503.0,-1092.0,-790.0,2.6473,579.088757,578.051873,569.137172,8.914701,10.832775,17.707207,0.0
606,2023-07-06,573.0,574.0,565.0,565.0,32070.0,563.0,-16476.0,-574.0,-603.0,-14045.0,5.1447,576.271005,576.043892,568.830715,7.213177,10.108856,7.697299,0.0
607,2023-07-07,565.0,572.0,563.0,565.0,19859.0,32.0,-4486.0,-185.0,-100.0,-1681.0,3.6264,574.016804,574.344832,568.546958,5.797874,9.246659,1.662877,1.0
608,2023-07-10,567.0,573.0,565.0,565.0,18996.0,-17.0,-329.0,-67.0,-775.0,-1141.0,3.9634,572.213443,572.907166,568.284221,4.622945,8.321917,0.333989,1.0
609,2023-07-11,574.0,577.0,570.0,577.0,18567.0,-263.0,9680.0,173.0,-306.0,4659.0,3.3345,573.170755,573.536832,568.929834,4.606998,7.578933,0.348810,1.0
610,2023-07-12,574.0,578.0,572.0,578.0,16220.0,-107.0,5154.0,2.0,-1385.0,543.0,2.6385,574.136604,574.223474,569.601698,4.621776,6.987501,0.495048,1.0
611,2023-07-13,587.0,590.0,585.0,585.0,26878.0,-566.0,20423.0,-647.0,403.0,9753.0,3.3511,576.309283,575.881401,570.742313,5.139088,6.617819,5.108553,1.0
612,2023-07-14,589.0,591.0,587.0,591.0,24381.0,-276.0,14641.0,-173.0,204.0,9289.0,3.3484,579.247426,578.207339,572.242882,5.964457,6.487146,13.098743,1.0


In [16]:
len(Library)

602

In [17]:
Library['y_1'].sum()

298.0

1.3. Concate dataframe function

In [18]:
def concate_Lib_Pred(Library, Prediction, th): 
    """
    th=-1 為Library
    th=n 為第n個Pred concate進Lib
    """
    if th < 0:
        Lib_Pred_df = Library
    
    else:
        row_to_add = Prediction.iloc[th]
        Lib_Pred_df = pd.concat([Library, row_to_add.to_frame().T], ignore_index=True)
        
    # 這種concate方法會有非數值問題, 要這樣修正
    Lib_Pred_df[Library.columns.to_list()[1:]] = Lib_Pred_df[Library.columns.to_list()[1:]].apply(pd.to_numeric, errors='coerce')
    Lib_Pred_df['Date'] = pd.to_datetime(Lib_Pred_df['Date'])

    return Lib_Pred_df

In [19]:
Lib_Pred_df = concate_Lib_Pred(Library=Library, Prediction=Prediction, th=0)
Lib_Pred_df.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_1
598,2023-06-27,570.0,575.0,569.0,572.0,22447.0,-71.0,-3850.0,-2006.0,501.0,-168.0,4.2795,577.684527,575.991298,563.541594,12.449704,14.589924,45.390598,0.0
599,2023-06-28,579.0,579.0,571.0,574.0,18685.0,-113.0,-3437.0,-93.0,378.0,-1260.0,4.0474,576.947622,575.684944,564.316291,11.368654,13.945670,41.019391,1.0
600,2023-06-29,578.0,580.0,570.0,573.0,18046.0,28.0,-1355.0,-121.0,1407.0,-434.0,3.7922,576.158098,575.271876,564.959529,10.312347,13.219005,34.887970,0.0
601,2023-06-30,570.0,576.0,568.0,576.0,33831.0,117.0,-2573.0,38737.0,649.0,-1545.0,6.1965,576.126478,575.383895,565.777341,9.606554,12.496515,34.306232,1.0
602,2023-07-03,578.0,580.0,576.0,579.0,15118.0,97.0,2353.0,-305.0,1401.0,582.0,2.5477,576.701182,575.940219,566.756798,9.183421,11.833896,33.311646,1.0


1.3.Data normalize function

In [20]:
def data_normalize(Lib_Pred_df):

    feature_to_standardize = Lib_Pred_df.columns.to_list()
    feature_to_standardize.remove(Lib_Pred_df.columns[0]) # 排除Date
    feature_to_standardize.remove(Lib_Pred_df.columns[-1])

    scaler = MinMaxScaler() 
    Lib_Pred_df[feature_to_standardize] = scaler.fit_transform(Lib_Pred_df[feature_to_standardize])

    return Lib_Pred_df

In [21]:
Lib_Pred_df = data_normalize(Lib_Pred_df)
Lib_Pred_df.tail()

,Date,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14,y_1
598,2023-06-27,0.634340,0.634304,0.646104,0.644231,0.103541,0.505087,0.428126,0.198268,0.332064,0.468425,0.138160,0.702469,0.707008,0.676538,0.624890,0.733508,0.453906,0.0
599,2023-06-28,0.662957,0.647249,0.652597,0.650641,0.076768,0.498513,0.430917,0.228551,0.324855,0.459063,0.127218,0.699748,0.705846,0.679815,0.605314,0.720099,0.410194,1.0
600,2023-06-29,0.659777,0.650485,0.649351,0.647436,0.072220,0.520582,0.444985,0.228107,0.385161,0.466145,0.115186,0.696833,0.704279,0.682536,0.586186,0.704976,0.348880,0.0
601,2023-06-30,0.634340,0.637540,0.642857,0.657051,0.184557,0.534512,0.436755,0.843222,0.340737,0.456620,0.228538,0.696716,0.704704,0.685995,0.573406,0.689939,0.343062,1.0
602,2023-07-03,0.659777,0.650485,0.668831,0.666667,0.051382,0.531382,0.470040,0.225195,0.384809,0.474855,0.056513,0.698838,0.706814,0.690138,0.565743,0.676148,0.333116,1.0


# MDRSmap I

2.1. 製作可餵入EDM格式的train_feature

In [22]:
def find_train_target_feature(data, target):

    df_columns = list(data.columns)
    train_feature = df_columns.copy()
    train_feature.remove('Date') # 先拿掉日期
    
    formatted_columns = ' '.join(df_columns[1:]) # 變成可以餵給 EDM function 參數 'columns' 的形式 
    train_feature.remove(target) # 再拿掉 target_feature

    return formatted_columns, train_feature

In [23]:
formatted_columns, train_feature = find_train_target_feature(data=Library, target=TARGET)
formatted_columns

'open high low close volume financing fi ii di rp capital EMA9 EMA12 EMA26 MACD Signal RSI14 y_1'

2.2. 找出target_feature最佳嵌入維度

In [24]:
def find_target_OED(data, target):

    target_OED = EmbedDimension(dataFrame=data, lib=f'1 {len(data)}', pred=f'{len(data)-21} {len(data)-1}', columns=target, showPlot=False) # 4. lib訓練全部, pred看最後20筆

    target_OED_rho = target_OED['rho'].max()
    target_OED = int(target_OED['E'][target_OED['rho'] == target_OED['rho'].max()].iloc[0])

    return target_OED, target_OED_rho


In [25]:
target_OED, target_OED_rho = find_target_OED(data=Library, target=TARGET)
print(f'target_OED: {target_OED}, target_OED_rho: {target_OED_rho}')

target_OED: 10, target_OED_rho: 0.25084041819504455


2.3. 找出所有有因果關係的train_feature

In [26]:
def find_rho_sig_df(data, ticker, target, target_OED, train_feature, E_max):
    
    crirho = stats.t.ppf(0.95, len(data) - 1) / (len(data) - 2 + stats.t.ppf(0.95, len(data) - 1) ** 2)
    ccm_libSizes = f'{target_OED+10} {len(data)-10} 10'
    # ccm_libSizes = list(range(10, len(data) + 1, 10)) + [len(data)]  # sequence of library size # original

    rho_sig_df = pd.DataFrame(columns=train_feature)
    for train in train_feature:

        ### 找出該train_feature最好的ccm_OED ###
        ccm_E_termRHO = pd.DataFrame(columns=['E', 'term_rho'])
        """
        這裡假設用 term_rho 來選 ccm_OED
        """
        for e in range(1, E_max+1):
            ccm_result = CCM(dataFrame=data, E=e, columns=train, target=target,
                            libSizes=ccm_libSizes, random=False, showPlot=False)
            # print(e, ccm_result[f'{target}:open'].iloc[-1]) # 有時候會有warning, 測試用
            new_data = {'E': e, 'term_rho': ccm_result[f'{target}:{train}'].iloc[-1]}
            ccm_E_termRHO.loc[len(ccm_E_termRHO)] = new_data

        max_term_rho_index = ccm_E_termRHO['term_rho'].idxmax()
        ccm_OED = ccm_E_termRHO.at[max_term_rho_index, 'E']

        ### 用最好的ccm_OED來做該feature的因果檢定 ###
        ccm_result = CCM(dataFrame=data, E=ccm_OED, columns=train, target=target, 
                        libSizes=ccm_libSizes, random=False, showPlot=False)
        """
        這裡假設用 target:train 、 LibSize 來做 kendalltau 檢定
        """
        ccm_result = ccm_result[['LibSize', f'{target}:{train}']]
        ccm_result[f'{target}:{train}'][ccm_result[f'{target}:{train}'] < 0] = 0
        term_rho = ccm_result[f'{target}:{train}'].iloc[-1]

        tau, p_value = kendalltau(ccm_result['LibSize'], ccm_result[f'{target}:{train}']) # 進行 kendalltau 相關檢定

        alpha = 0.05
        if (p_value < alpha) and (term_rho > crirho): # 顯著相關
            rho_sig_df[train] = [term_rho]

        else: # "不" 顯著相關
            rho_sig_df[train] = [0]

    rho_sig_df.index = pd.Index([f'{ticker}_{target}']) 
    
    return rho_sig_df


In [27]:
rho_sig_df = find_rho_sig_df(data=Library, ticker=TICKER, target=TARGET, 
                             target_OED=target_OED, train_feature=train_feature, E_max=10)
rho_sig_df

,open,high,low,close,volume,financing,fi,ii,di,rp,capital,EMA9,EMA12,EMA26,MACD,Signal,RSI14
2330_y_1,0.185016,0.1981,0.182244,0.230233,0.146545,0,0,0,0,0.480324,0.111742,0.223015,0.245364,0.335423,0,0,0.396142


2.4. 用有因果關係的train_feature建立Embed_df

In [28]:
def make_Embed_df(data, max_lag, target, rho_sig_df):

    #用有因果關係的 train feature + target feature 製作 Embed_df #
    non_zero_columns = rho_sig_df.loc[:, (rho_sig_df != 0).any(axis = 0)] # 選取值非0的column
    train_feature_ls = list(non_zero_columns.columns)
    formatted_columns = ' '.join(train_feature_ls) # 轉成 EDM column 的 input
    columns_to_lag = formatted_columns + f' {target}' # 加入 target 本身

    Embed_df = Embed(dataFrame=data, E=max_lag, tau=-1, columns=columns_to_lag) # 製作 Embed_df
    Embed_df['Date'] = data['Date'] # 加入Date來看index, 才可以防simplex func的bug
    Embed_df.dropna(inplace=True) # 把包含NaN的資料拿掉
    Embed_df = Embed_df.reset_index(drop=True)
    Embed_df = Embed_df[['Date'] + [col for col in Embed_df.columns if col != 'Date']]

    ML_df_date = Embed_df.copy()
    ML_df_date['Date'] = pd.to_datetime(ML_df_date['Date']) # 將index設為日期
    ML_df_date.set_index('Date', inplace=True)
    ML_df_date = ML_df_date.filter(like="(t-0)") # 只留下(t-0)的column

    return Embed_df, ML_df_date

In [29]:
Embed_df, ML_df_date = make_Embed_df(data=Lib_Pred_df, max_lag=10, target=TARGET, rho_sig_df=rho_sig_df)
Embed_df.tail()

,Date,open(t-0),open(t-1),open(t-2),open(t-3),open(t-4),open(t-5),open(t-6),open(t-7),open(t-8),...,y_1(t-0),y_1(t-1),y_1(t-2),y_1(t-3),y_1(t-4),y_1(t-5),y_1(t-6),y_1(t-7),y_1(t-8),y_1(t-9)
589,2023-06-27,0.634340,0.653418,0.672496,0.662957,0.675676,0.697933,0.697933,0.697933,0.707472,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
590,2023-06-28,0.662957,0.634340,0.653418,0.672496,0.662957,0.675676,0.697933,0.697933,0.697933,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
591,2023-06-29,0.659777,0.662957,0.634340,0.653418,0.672496,0.662957,0.675676,0.697933,0.697933,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
592,2023-06-30,0.634340,0.659777,0.662957,0.634340,0.653418,0.672496,0.662957,0.675676,0.697933,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
593,2023-07-03,0.659777,0.634340,0.659777,0.662957,0.634340,0.653418,0.672496,0.662957,0.675676,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [30]:
ML_df_date.tail()

,open(t-0),high(t-0),low(t-0),close(t-0),volume(t-0),rp(t-0),capital(t-0),EMA9(t-0),EMA12(t-0),EMA26(t-0),RSI14(t-0),y_1(t-0)
Date,,,,,,,,,,,,
2023-06-27,0.634340,0.634304,0.646104,0.644231,0.103541,0.468425,0.138160,0.702469,0.707008,0.676538,0.453906,0.0
2023-06-28,0.662957,0.647249,0.652597,0.650641,0.076768,0.459063,0.127218,0.699748,0.705846,0.679815,0.410194,1.0
2023-06-29,0.659777,0.650485,0.649351,0.647436,0.072220,0.466145,0.115186,0.696833,0.704279,0.682536,0.348880,0.0
2023-06-30,0.634340,0.637540,0.642857,0.657051,0.184557,0.456620,0.228538,0.696716,0.704704,0.685995,0.343062,1.0
2023-07-03,0.659777,0.650485,0.668831,0.666667,0.051382,0.474855,0.056513,0.698838,0.706814,0.690138,0.333116,1.0


2.5. 用simplex randomsearch找出最佳的view

In [31]:
def make_random_simplex(Embed_df, target, target_OED, kmax, kn):    

    Embed_for_train = Embed_df.drop(columns='Date') # 先把 Date 拿掉
    Embed_for_train = Embed_for_train.drop(columns=f'{target}(t-0)') # 先把 target 拿掉
    train_f_ls = list(Embed_for_train.columns) # train_feature
    train_f_num = len(Embed_for_train.columns) # train_feature 的個數

    rho_feature_view = pd.DataFrame(columns=['rho']) # 創建一個df去紀錄每個隨機view的資料
    new_column = pd.DataFrame(columns=['feature_' + str(i) for i in range(1, target_OED+1)])
    rho_feature_view = pd.concat([rho_feature_view, new_column], axis=1)
    k = 1
    while k <= kmax:
        random_pick_train = np.random.choice(train_f_num, target_OED, replace=False)
        # print(random_pick_train)

        train_f_ls = np.array(train_f_ls) # 變成 array 才可以一次選
        select_train_f = train_f_ls[random_pick_train] # 隨機選到的 train_feature
        formatted_random_columns = ' '.join(select_train_f) # 用成符合 EDM 的資料格式
        # print(formatted_random_columns)

        simp = Simplex(dataFrame=Embed_df, E=target_OED, # ver3: 測試近10 or 20個交易日
                       lib=f'1 {len(Embed_df)}', pred = f'{len(Embed_df)-21} {len(Embed_df)-1}', 
                       columns=formatted_random_columns, target=f'{target}(t-0)',
                       embedded = True, showPlot = False) # 原本是False現在改True
        # print(simp)

        sub_simp = simp[['Observations', 'Predictions']] # 計算rho
        rho = sub_simp['Observations'].corr(sub_simp['Predictions'])

        rho_feature_view.loc[len(rho_feature_view), 'rho'] = rho # 將 view 更新到 rho_feature_view 的 df 中
        rho_feature_view.loc[len(rho_feature_view)-1, rho_feature_view.columns[1:]] = select_train_f
        # print(rho)
        k += 1

    allscore = rho_feature_view.sort_values(by='rho', ascending=False).head(kn)
    allscore = allscore.reset_index(drop=True)

    return allscore

In [32]:
allscore = make_random_simplex(Embed_df=Embed_df, target=TARGET, target_OED=target_OED, kmax=10000, kn=5)
allscore.head()

,rho,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10
0,0.78534,open(t-6),high(t-5),high(t-0),y_1(t-7),EMA9(t-6),EMA12(t-2),low(t-8),open(t-1),EMA12(t-6),EMA26(t-8)
1,0.775585,open(t-6),EMA26(t-1),open(t-2),EMA26(t-6),low(t-9),EMA12(t-6),y_1(t-7),high(t-6),high(t-1),volume(t-4)
2,0.771036,EMA26(t-7),y_1(t-7),high(t-9),open(t-0),volume(t-4),EMA12(t-3),EMA26(t-2),open(t-5),low(t-6),EMA26(t-0)
3,0.756388,EMA26(t-8),EMA26(t-7),EMA26(t-5),EMA12(t-0),rp(t-2),y_1(t-7),EMA26(t-6),close(t-8),capital(t-2),EMA9(t-7)
4,0.754763,open(t-3),y_1(t-7),EMA26(t-9),low(t-5),EMA26(t-7),high(t-5),low(t-6),EMA9(t-4),EMA9(t-3),EMA12(t-6)


# MDRSmap II

3.1. 計算每個時點的(view加權)距離

In [33]:
def compute_view_w_distance(Embed_df, allscore):

    ww = allscore['rho'] / allscore['rho'].sum() # 每個view的權重

    dmatrix_ls = []
    for j in range(allscore.shape[0]):

        view_feature = allscore.iloc[j, 1:] # 選取第j個view的所有feature
        view_feature = np.array(view_feature) # 把所有feature變成array才可以從完整Embed_df中找資料
        view_feature_value = Embed_df[view_feature]
        view_matrix = view_feature_value.to_numpy() # 從df形式變array
        view_matrix = np.vstack(view_matrix) # 這樣才能疊成matrix

        Dx_t2 = pdist(view_matrix, metric='euclidean') * ww[j] # 計算加權距離
        Dx_t2 = squareform(Dx_t2) # 將距離變成squareform
        dmatrix_ls.append(Dx_t2)

    v_w_dmatrix = np.sum(dmatrix_ls, axis=0) # 輸出每個時點的view加權距離

    return v_w_dmatrix

In [34]:
v_w_dmatrix = compute_view_w_distance(Embed_df=Embed_df, allscore=allscore)
v_w_dmatrix

array([[0.        , 0.17744251, 0.26555083, ..., 1.07674886, 1.07717467,
        0.40123522],
       [0.17744251, 0.        , 0.14997336, ..., 1.0805679 , 1.07841368,
        0.41484388],
       [0.26555083, 0.14997336, 0.        , ..., 1.08146109, 1.07829127,
        0.41799859],
       ...,
       [1.07674886, 1.0805679 , 1.08146109, ..., 0.        , 0.03677619,
        1.00081519],
       [1.07717467, 1.07841368, 1.07829127, ..., 0.03677619, 0.        ,
        1.00112558],
       [0.40123522, 0.41484388, 0.41799859, ..., 1.00081519, 1.00112558,
        0.        ]])

3.2.尋找elastic-net最佳參數

In [26]:
# ### test ###
# target = 'bs'
# Tp=1

# ML_df_date_new = ML_df_date.copy()
# ML_df_date_new[f'ans(t-0)'] = ML_df_date_new[f'{target}(t-0)'].shift(-Tp) # step.1: 先將target往前移Tp, 製作y
# # ML_df_date_new = ML_df_date_new.multiply(w_tp, axis=0) # step.2: 再將data乘上距離加權
# ML_df_date_new = ML_df_date_new[:-(Tp+1)] # step.3: 拿掉最後Tp+1個, 因為最後面的data是硬拼上去的
# ML_df_date_new

In [35]:
def find_MDRSmap_param(target, ML_df_date, theta_seq, v_w_dmatrix, Tp):

    result_ls = pd.DataFrame(columns=['Theta', 'Score', 'Param']) # 創建紀錄回測結果的dataframe

    ### 將原始資料乘上空間位置權數 ###
    tp = len(ML_df_date) -1
    tp_distence = v_w_dmatrix[tp] # 第tp個時點離其他時點的距離
    mask = np.ones(len(tp_distence), dtype=bool) # 遮蔽該時點計算平均數
    mask[tp] = False
    dpar = np.mean(tp_distence[mask]) # 第tp個時點離其他時點的平均數

    for theta in theta_seq:
        w_tp = np.exp(-theta * tp_distence / dpar) # 計算每個時點資料的加權
        w_tp = np.sqrt(w_tp)

        ### 加入答案列 ###
        ML_df_date_new = ML_df_date.copy()
        ML_df_date_new[f'ans(t-0)'] = ML_df_date_new[f'{target}(t-0)'].shift(-Tp) # step.1: 先將target往前移Tp, 製作y
        ML_df_date_new = ML_df_date_new.multiply(w_tp, axis=0) # step.2: 再將data乘上距離加權
        ML_df_date_new['ans(t-0)'] = ML_df_date_new['ans(t-0)'].apply(lambda x: 1.0 if x != 0 else x) # step.3: 把ans非0的部分變成1
        ML_df_date_new = ML_df_date_new[:-(Tp+1)] # step.4: 拿掉最後Tp+1個, 因為最後面的data是硬拼上去的
        # ML_df_date_new = ML_df_date_new.drop(columns=[f'{target}(t-0)']) # step.5: 原paper有刪target啦, 這邊可選擇刪或不刪
        # ML_df_date_new = ML_df_date_new.dropna().reset_index(drop=True) # 不確定要不要用

        ### 分拆train, validation(以近60天為基準) ###
        X = ML_df_date_new.iloc[:, :-1]
        y = ML_df_date_new.iloc[:, -1]
        val_fold = [-1] * (len(X)-60) + [0] * 60 # 最後60筆當validation set
        ps = PredefinedSplit(test_fold=val_fold)

        logistic_elastic_net = LogisticRegression(penalty='elasticnet', 
                                                  solver='saga', # 只有saga支持elasticnet
                                                  random_state=87)

        ### grid search ###
        param_grid = {'l1_ratio': [0.9, 0.1, 0.01, 0.001, 0.0001],
                      'C': [0.001, 0.01, 0.1, 1, 10],
                      'tol': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
                      'fit_intercept': [True], 
                      'intercept_scaling': [0.1],
                      'warm_start': [True]}
        grid_search = GridSearchCV(estimator=logistic_elastic_net, 
                                   param_grid=param_grid, 
                                   cv=ps, scoring='accuracy', 
                                   return_train_score=True)       

        grid_search.fit(X, y)

        ### 記錄結果 ###
        result_ls.loc[len(result_ls), 'Theta'] = theta
        result_ls.loc[len(result_ls)-1, 'Score'] = grid_search.best_score_
        result_ls.loc[len(result_ls)-1, 'Param'] = [grid_search.best_params_]

        theta = result_ls['Theta'][result_ls['Score'].idxmax()]
        param = result_ls['Param'][result_ls['Score'].idxmax()][0]

    return result_ls, theta, param

In [36]:
result_ls, theta, param = find_MDRSmap_param(target=TARGET, 
                                             ML_df_date=ML_df_date, 
                                             theta_seq=[1,2,4,7,11,16,22], 
                                             v_w_dmatrix=v_w_dmatrix,
                                             Tp=TP)

In [37]:
result_ls['Param'][5]

[{'C': 0.001,
  'fit_intercept': True,
  'intercept_scaling': 0.1,
  'l1_ratio': 0.9,
  'tol': 1e-05,
  'warm_start': True}]

3.3.用最佳參數訓練MDRSmap

In [38]:
def MDRSmap_model(target, ML_df_date, theta, v_w_dmatrix, param, Tp):

    ### 將原始資料乘上空間位置權數 ###
    tp = len(ML_df_date) -1
    tp_distence = v_w_dmatrix[tp] # 第tp個時點離其他時點的距離
    mask = np.ones(len(tp_distence), dtype=bool) # 遮蔽該時點計算平均數
    mask[tp] = False
    dpar = np.mean(tp_distence[mask]) # 第tp個時點離其他時點的平均數

    w_tp = np.exp(-theta * tp_distence / dpar) # 計算每個時點資料的加權
    w_tp = np.sqrt(w_tp)

    ### 加入答案列 ###
    ML_df_date_new = ML_df_date.copy()
    ML_df_date_new[f'ans(t-0)'] = ML_df_date_new[f'{target}(t-0)'].shift(-Tp) # step.1: 先將target往前移Tp, 製作y
    ML_df_date_new = ML_df_date_new.multiply(w_tp, axis=0) # step.2: 再將data乘上距離加權
    ML_df_date_new['ans(t-0)'] = ML_df_date_new['ans(t-0)'].apply(lambda x: 1.0 if x != 0 else x) # step.3: 把ans非0的部分變成1
    ML_df_date_new = ML_df_date_new[:-(Tp+1)] # step.4: 拿掉最後Tp+1個, 因為最後面的data是硬拼上去的
    # ML_df_date_new = ML_df_date_new.drop(columns=[f'{target}(t-0)']) # step.5: 原paper有刪target啦, 這邊可選擇刪或不刪
    # ML_df_date_new = ML_df_date_new.dropna().reset_index(drop=True) # 不確定要不要用

    ### 分拆train, validation(以近60天為基準) ###
    X = ML_df_date_new.iloc[:, :-1]
    y = ML_df_date_new.iloc[:, -1]

    logistic_elastic_net = LogisticRegression(penalty='elasticnet', 
                                              solver='saga', # 只有saga支持elasticnet
                                              random_state=87,
                                              **param)
                             

    logistic_elastic_net.fit(X, y)

    return logistic_elastic_net

In [39]:
logistic_elastic_net = MDRSmap_model(target=TARGET, ML_df_date=ML_df_date, 
                                     theta=theta, v_w_dmatrix=v_w_dmatrix, param=param, Tp=TP)

3.4.進行預測

In [40]:
X_pred = np.array(ML_df_date.iloc[-1]).reshape(1, -1)
y_pred = logistic_elastic_net.predict(X_pred)
y_pred = y_pred[0]
y_pred

0.0

3.5.製作評估dataframe

In [33]:
# Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-11-30')].reset_index(drop=True)
# Today = origi_data['bs'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-11-30')].reset_index(drop=True)
# Yesterday = origi_data['bs'][(origi_data['Date']>='2023-06-30')&(origi_data['Date']<='2023-11-29')].reset_index(drop=True)

# MDRSmap_result = pd.DataFrame(Date)
# MDRSmap_result['Observations'] = Today
# MDRSmap_result['Predictions'] = None
# MDRSmap_result['Yesterday'] = Yesterday
# MDRSmap_result

In [34]:
# th=0
# MDRSmap_result.loc[th, 'Predictions'] = y_pred
# MDRSmap_result

In [35]:
# MDRSmap_result['Date'][th]

### 四、完整預測流程

4.1.完整預測流程

In [41]:
### 從 6/30 預測 7/03 開始 ###

### 製作評估dataframe ###
Date = origi_data['Date'][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)
Today = origi_data[TARGET][(origi_data['Date']>='2023-07-01')&(origi_data['Date']<='2023-10-31')].reset_index(drop=True)
# Yesterday = origi_data[TARGET][(origi_data['Date']>='2023-06-30')&(origi_data['Date']<='2023-10-30')].reset_index(drop=True)

MDRSmap_result = pd.DataFrame(Date)
MDRSmap_result['Observations'] = Today
MDRSmap_result['Predictions'] = None
# MDRSmap_result['Yesterday'] = Yesterday

### 開始進行MDRSmap演算法 ###
for th in range(0, len(Prediction)):

    if th == 0: ### th = 0, 要找view ###
        Lib_Pred_df = concate_Lib_Pred(Library=Library, Prediction=Prediction, th=th)
        Lib_Pred_df = data_normalize(Lib_Pred_df)
        formatted_columns, train_feature = find_train_target_feature(data=Lib_Pred_df, target=TARGET)
        target_OED, target_OED_rho = find_target_OED(data=Lib_Pred_df, target=TARGET)
        rho_sig_df = find_rho_sig_df(data=Lib_Pred_df, ticker=2330, target=TARGET, 
                                     target_OED=target_OED, train_feature=train_feature, E_max=10)
        Embed_df, ML_df_date = make_Embed_df(data=Lib_Pred_df, max_lag=10, target=TARGET, rho_sig_df=rho_sig_df)
        allscore = make_random_simplex(Embed_df=Embed_df, target=TARGET, target_OED=target_OED, kmax=10000, kn=5)
        v_w_dmatrix = compute_view_w_distance(Embed_df=Embed_df, allscore=allscore)
        result_ls, theta, param = find_MDRSmap_param(target=TARGET, 
                                                     ML_df_date=ML_df_date, 
                                                     theta_seq=[1,2,4,7,11,16,22], 
                                                     v_w_dmatrix=v_w_dmatrix,
                                                     Tp=TP)
        logistic_elastic_net = MDRSmap_model(target=TARGET, ML_df_date=ML_df_date, 
                                             theta=theta, v_w_dmatrix=v_w_dmatrix, param=param, Tp=TP)
        
        ### 預測 ###
        X_pred = np.array(ML_df_date.iloc[-1]).reshape(1, -1)
        y_pred = logistic_elastic_net.predict(X_pred)
        y_pred = y_pred[0]

        ### 將結果併入評估dataframe ###
        MDRSmap_result.loc[th, 'Predictions'] = y_pred
        print(f"{MDRSmap_result['Date'][th]}: finished")
    
    else: ### th > 0, 僅算距離+預測 ###
        Lib_Pred_df = concate_Lib_Pred(Library=Library, Prediction=Prediction, th=th)
        Lib_Pred_df = data_normalize(Lib_Pred_df)
        Embed_df, ML_df_date = make_Embed_df(data=Lib_Pred_df, max_lag=10, target=TARGET, rho_sig_df=rho_sig_df)
        v_w_dmatrix = compute_view_w_distance(Embed_df=Embed_df, allscore=allscore)
        result_ls, theta, param = find_MDRSmap_param(target=TARGET, 
                                                     ML_df_date=ML_df_date, 
                                                     theta_seq=[1,2,4,7,11,16,22], 
                                                     v_w_dmatrix=v_w_dmatrix,
                                                     Tp=TP)
        logistic_elastic_net = MDRSmap_model(target=TARGET, ML_df_date=ML_df_date, 
                                             theta=theta, v_w_dmatrix=v_w_dmatrix, param=param, Tp=TP)
        
        ### 預測 ###
        X_pred = np.array(ML_df_date.iloc[-1]).reshape(1, -1)
        y_pred = logistic_elastic_net.predict(X_pred)
        y_pred = y_pred[0]

        ### 將結果併入評估dataframe ###
        MDRSmap_result.loc[th, 'Predictions'] = y_pred
        print(f"{MDRSmap_result['Date'][th]}: finished")

2023-07-03: finished
2023-07-04: finished
2023-07-05: finished
2023-07-06: finished
2023-07-07: finished
2023-07-10: finished
2023-07-11: finished
2023-07-12: finished
2023-07-13: finished
2023-07-14: finished
2023-07-17: finished
2023-07-18: finished
2023-07-19: finished
2023-07-20: finished


4.3.評估結果

In [42]:
ACC = len(MDRSmap_result[MDRSmap_result['Predictions'] == MDRSmap_result['Observations']]) / len(MDRSmap_result['Observations'])
print('ACC: ', ACC)

ACC:  0.12048192771084337


In [43]:
MDRSmap_result.head(60)

,Date,Observations,Predictions
0,2023-07-03,1,0.0
1,2023-07-04,0,0.0
2,2023-07-05,0,0.0
3,2023-07-06,1,0.0
4,2023-07-07,1,1.0
5,2023-07-10,1,0.0
6,2023-07-11,1,1.0
7,2023-07-12,1,1.0
8,2023-07-13,1,1.0
9,2023-07-14,1,1.0
